In [1]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('../..'))) # access sibling directories
sys.path.append("C:\\Users\\lasse\\Documents\\GitHub\\BayesFlow")

from src.python.settings import summary_meta_diffusion, probability_meta_diffusion
from src.python.helpers import load_simulated_rt_data, mask_inputs
from src.python.networks import HierarchicalInvariantNetwork, ModelProbabilityNetwork
from src.python.losses import softmax_loss

import numpy as np
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.optimizers import Adam
from functools import partial

from bayesflow.trainers import ModelComparisonTrainer
from bayesflow.amortizers import MultiModelAmortizer 

# Pre-training with 100 trials per person

## Load in data

In [2]:
levy_sims_folder = "c:\\Users\\lasse\\documents\\hierarchical_model_comparison_project\\data\\03_levy_flight_application\\truncnormal_alpha_prior"

indices_100_filename = "train_indices_100_trials.npy"
datasets_100_filename = "train_datasets_100_trials.npy"

indices_100, datasets_100 = load_simulated_rt_data(levy_sims_folder, indices_100_filename, datasets_100_filename)

In [3]:
datasets_100.shape

(40000, 40, 100, 3)

## Training

When retraining: Move to new trial folder so that networks are initialized from scratch.

In [4]:
summary_net = HierarchicalInvariantNetwork(summary_meta_diffusion)
probability_net = ModelProbabilityNetwork(probability_meta_diffusion)
amortizer = MultiModelAmortizer(probability_net, summary_net)

In [5]:
# Training steps
epochs = 20 
n_datasets = datasets_100.shape[0]
batch_size = 32
iterations_per_epoch=n_datasets/batch_size

# Cosine decaying learning rate
initial_lr = 0.0005
decay_steps = epochs*iterations_per_epoch
alpha = 0
lr_schedule = CosineDecay(
    initial_lr, decay_steps, alpha=alpha
    )


# Checkpoint path for storing pretrained network
checkpoint_path = "c:\\Users\\lasse\\documents\\hierarchical_model_comparison_project\\checkpoints\\03_levy_flight_application\\truncnormal_alpha_prior\\pre-trained_net"

trainer = ModelComparisonTrainer(
    network=amortizer, 
    loss=partial(softmax_loss),
    optimizer=partial(Adam, lr_schedule),
    checkpoint_path=checkpoint_path,
    skip_checks=True
    )

TRAINER INITIALIZATION: No generative model provided. Only offline learning mode is available!
Initializing networks from scratch.


In [6]:
losses = trainer.train_offline(epochs=epochs, batch_size=batch_size, 
                               model_indices=indices_100, sim_data=datasets_100)

Converting 40000 simulations to a TensorFlow data set...


Training epoch 1:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 2:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 3:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 4:   0%|          | 0/1250 [00:00<?, ?it/s]

...

Training epoch 16:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 17:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 18:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 19:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 20:   0%|          | 0/1250 [00:00<?, ?it/s]

SOFTMAX
30 epochs
- training time = 39 minutes
- running loss after 20 epochs = 0.2

- training time =  minutes
- running loss after 20 epochs = 0.12

OLD EVIDENTIAL STUFF
results for trial 1 / trial 2
- training time = 62 minutes / 64 minutes
- running loss after 32 epochs = 0.469 / 0.462
results for run with new alpha prior
- training time = 67 minutes
- running loss after 32 epochs = 0.343